In [3]:
from bs4 import BeautifulSoup
import json
from urllib.parse import urljoin
import re
import requests
from collections import defaultdict

class Coletor:
    def __init__(self):
        self.urls_visitadas = set()
        self.dados_coletados = []

    def coletar_urls(self, url_inicial, profundidade=1):
        if profundidade == 0 or url_inicial in self.urls_visitadas:
            return

        self.urls_visitadas.add(url_inicial)
        html = self.buscar_html(url_inicial)

        if html:
            soup = BeautifulSoup(html, 'html.parser')
            self.dados_coletados.append({'url': url_inicial})

            for link in soup.find_all('a', href=True):
                href = link['href']
                href_absoluto = urljoin(url_inicial, href)
                if self.validar_url(href_absoluto):
                    self.coletar_urls(href_absoluto, profundidade - 1)

    def buscar_html(self, url):
        try:
            resposta = requests.get(url, allow_redirects=True)
            if resposta.status_code == 200:
                return resposta.text
        except requests.exceptions.RequestException as erro:
            print(f"Erro ao buscar HTML da URL {url}: {erro}")
        return None

    def validar_url(self, url):
        return re.match(r'^https://www.ufop.br/', url)

    def salvar_dados_json(self, nome_arquivo):
        with open(nome_arquivo, 'w') as arquivo:
            json.dump(self.dados_coletados, arquivo, indent=4)

class Indexador:
    def __init__(self):
        self.indice_invertido = defaultdict(list)

    def indexar_pagina(self, url, texto):
        if "licenciamentoambiental" in texto.lower():
            print(f'Palavra "licenciamentoambiental" encontrada na página: {url}')
            self.indice_invertido[url].append("licenciamentoambiental")

if __name__ == "__main__":
    urls_iniciais = ["https://www.ufop.br/"]
    coletor = Coletor()
    indexador = Indexador()

    for url in urls_iniciais:
        coletor.coletar_urls(url, profundidade=3)

    coletor.salvar_dados_json('dados_coletados.json')

    for dado in coletor.dados_coletados:
        url = dado['url']
        html = coletor.buscar_html(url)
        if html:
            indexador.indexar_pagina(url, html)


Palavra "licenciamentoambiental" encontrada na página: https://www.ufop.br/
Palavra "licenciamentoambiental" encontrada na página: https://www.ufop.br/#page-content
Palavra "licenciamentoambiental" encontrada na página: https://www.ufop.br/#header
Palavra "licenciamentoambiental" encontrada na página: https://www.ufop.br/#menu
Palavra "licenciamentoambiental" encontrada na página: https://www.ufop.br/#content
Palavra "licenciamentoambiental" encontrada na página: https://www.ufop.br/#footer
Palavra "licenciamentoambiental" encontrada na página: https://www.ufop.br/#off-canvas
Palavra "licenciamentoambiental" encontrada na página: https://www.ufop.br/alunos-graduacao
Palavra "licenciamentoambiental" encontrada na página: https://www.ufop.br/alunos-pos-graduacao
Palavra "licenciamentoambiental" encontrada na página: https://www.ufop.br/tae
Palavra "licenciamentoambiental" encontrada na página: https://www.ufop.br/docentes
Palavra "licenciamentoambiental" encontrada na página: https://www